In [1]:
NAME = "Sarale Goldberger"
COLLABORATORS = ""

# Instructions

1. Make sure you have filled out your "NAME" and "COLLABORATORS" (if any) in the previous cell.

2. You should complete all code/markdown cells that state "YOUR CODE HERE" or "YOUR ANSWER HERE". 
   
3. Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

4. Partial credit can be obtained if your solution approach is clear and the documented within comments in the implementation.

5. You should follow good coding practices. Your code should use type hints, be robust against invalid inputs, and you should also write a few test cases to check for correctness particularly including edge cases.  


## Problem 1

Write a python function which computes third power of a number if the number is odd, and the square of the number if the number is even.

In [2]:
# the function takes in an int x and returns an int
def parity_power(x: int) -> int:
    if x % 2 == 0:
        return x**2
    return x**3
    raise NotImplementedError()

In [3]:
# provide test cases
assert parity_power(1) == 1
assert parity_power(2) == 4
assert parity_power(3) == 27
assert parity_power(24) == 24**2
assert parity_power(377) == 377**3
print("Your function works!")

Your function works!


## Problem 2

An important part of this course is to complete an ML based project over the course of the semester.  We are going to start right now!

Read the description of the project uploaded on canvas and write down 2 potential ideas for your project.  

I submitted my ideas in the 'Term Project Ideas' assignment in Canvas. My idea is to use my NLP research with Professor Waxman. I will be taking audio recordings of classes which have a mix of English, Hebrew, Yiddish, Arameic, and Yeshivish slang words, running them through WhisperAI to obtain preliminary transcriptions, and then using prompt strategies to train ChatGPT to fix the mistakes in the non-English words.

## Problem 3

This problem will test some of your programming skills and data wrangling ability.  

We will start with a dataset from 'https://github.com/JeffSackmann/tennis_atp' which covers information on Tennis matches from the past few decades.  

This dataset is provided by Jeff Sackman.

Write a function *download_data_by_year* which takes the year as an input and returns a pandas dataframe with the data from the file of the form `atp_matches_{year}.csv`.  Implement a cache so that we do not download the data if it is already in the cache. 

**Hint**: The url you need to use to access the files in github is https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_{year}.csv

In [12]:
import pandas as pd

# cache for the df's that have already been downloaded
cache = {}

def download_data_by_year(year: int) -> pd.DataFrame:
    # Check if the input year has already been downloaded
    if year in cache:
        return cache[year]

    url = f"https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_{year}.csv"
    
    try:
        df = pd.read_csv(url)
        cache[year] = df
        return df
        
    except pd.errors.ParserError:
        print("Error parsing data")
    
    raise NotImplementedError()

In [13]:
df = download_data_by_year(2017)
assert df.shape == (2911, 49)

In [14]:
# Print part of the dataset
print("Input and target Features")
# display(pd.concat([df.data, df.target], axis=1).head())
df.info()
# df.filter(["surface", "draw_size", "tourney_level", "minutes"]).sort_values(by=['draw_size'])
# df.groupby(["tourney_level"]).size()

Input and target Features
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2911 entries, 0 to 2910
Data columns (total 49 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tourney_id          2911 non-null   object 
 1   tourney_name        2911 non-null   object 
 2   surface             2911 non-null   object 
 3   draw_size           2911 non-null   int64  
 4   tourney_level       2911 non-null   object 
 5   tourney_date        2911 non-null   int64  
 6   match_num           2911 non-null   int64  
 7   winner_id           2911 non-null   int64  
 8   winner_seed         1238 non-null   float64
 9   winner_entry        403 non-null    object 
 10  winner_name         2911 non-null   object 
 11  winner_hand         2910 non-null   object 
 12  winner_ht           2876 non-null   float64
 13  winner_ioc          2911 non-null   object 
 14  winner_age          2911 non-null   float64
 15  loser_id            2911 non-

## Problem 4
Download the data for years 2000-2020 (inclusive).  Compute the average number of matches played per year on each surface for each month. 

Return your solution as a dictionary where the key is the (surface, month) pair as a tuple and the value is the average.

**Hint**: You may have a case where no matches were played on a given surface in a month.  This should factor into your calculation as a zero.

In [15]:
def get_statistics(df):
    
    # compute the average number of matches played per year on each surface for each month
    df_grouped = df.groupby(['year', 'month', 'surface']).size().reset_index(name='matches')
    df_grouped_avg = df_grouped.groupby(['surface', 'month']).mean().round().astype(int).reset_index().fillna(0)
    print("df_grouped_avg\n", df_grouped_avg)
    ##print("Dimensions:", df_grouped_avg.shape)


    # create a dictionary where the key is the match, month pair as a tuple 
    # and the value is the average
    # there will be 12 * 4 keys for the 12 months * 4 surface types per month
    dict = {}
    months = [i for i in range(1,13)]
    surfaces = ['Carpet', 'Clay', 'Grass', 'Hard']
    i = 0
    for ind in range(len(df_grouped_avg)):
        mo = i%12 + 1  # expected month
        mo_df = df_grouped_avg['month'].iloc[ind]
        # if there are missing months add them in with data as 0
        if mo < mo_df:
            for j in range(mo, mo_df):
                dict[(df_grouped_avg['surface'].iloc[ind], j)] = 0
            i += mo_df - mo
        elif mo > mo_df:
            for j in range(mo, 13):
                dict[(df_grouped_avg['surface'].iloc[ind-1], j)] = 0
                i += 1
        # add the key-data pair of the current month and surface
        data = df_grouped_avg['matches'].iloc[ind]
        key = (df_grouped_avg['surface'].iloc[ind], mo_df)
        dict[key] = data
        # increment month tracker
        i+=1
    ##print(f"All months accounted for {i==len(months)*len(surfaces)}: i {i}, exp {len(months)*len(surfaces)}")
    return dict

    # raise NotImplementedError

In [16]:
# download the data for years 2000-2020 incl.
# initialize a df with the data from 2000, and concatenate the rest of the years
# add a year and month column, assuming 'tourney_date' column contains integers in the format YYYYMM
df = download_data_by_year(2000)      
df['year'] = 2000   
df['month'] = (df['tourney_date'] % 10000) // 100

for year in range(2001, 2021):
    # download the data
    temp = download_data_by_year(year)

    ## To check if there is a row with 5 for month and Carpet for surface.
    #print(year, ((df['month'] == 5) & (df['surface'] == 'Carpet')).any())
    
    # add year and month cols
    temp['year'] = year
    temp['month'] = (temp['tourney_date'] % 10000) // 100
    # concatenate the df to our larger df
    df = pd.concat([df, temp])

display(df)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year,month
0,2000-301,Auckland,Hard,32,A,20000110,1,103163,1.0,NaN,...,29.0,17.0,4.0,7.0,11.0,1612.0,63.0,595.0,2000,1
1,2000-301,Auckland,Hard,32,A,20000110,2,102607,NaN,Q,...,18.0,12.0,3.0,6.0,211.0,157.0,49.0,723.0,2000,1
2,2000-301,Auckland,Hard,32,A,20000110,3,103252,NaN,NaN,...,7.0,8.0,7.0,11.0,48.0,726.0,59.0,649.0,2000,1
3,2000-301,Auckland,Hard,32,A,20000110,4,103507,7.0,NaN,...,14.0,10.0,6.0,8.0,45.0,768.0,61.0,616.0,2000,1
4,2000-301,Auckland,Hard,32,A,20000110,5,102103,NaN,Q,...,18.0,12.0,5.0,9.0,167.0,219.0,34.0,873.0,2000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,2020-7485,Antwerp,Hard,32,A,20201019,125,105554,NaN,NaN,...,14.0,13.0,3.0,7.0,35.0,1384.0,79.0,802.0,2020,10
1458,2020-7485,Antwerp,Hard,32,A,20201019,126,200267,NaN,WC,...,10.0,12.0,4.0,9.0,528.0,58.0,45.0,1165.0,2020,10
1459,2020-7485,Antwerp,Hard,32,A,20201019,128,126203,7.0,NaN,...,5.0,9.0,3.0,5.0,28.0,1670.0,33.0,1402.0,2020,10
1460,2020-7485,Antwerp,Hard,32,A,20201019,129,144750,NaN,Q,...,5.0,10.0,7.0,11.0,90.0,748.0,74.0,838.0,2020,10


In [17]:
from numpy.testing import assert_allclose
res = get_statistics(df)
assert_allclose(res[('Carpet', 1)], 27)

df_grouped_avg
    surface  month  year  matches
0   Carpet      1  2005       27
1   Carpet      2  2006       19
2   Carpet      3  2010       13
3   Carpet      4  2005       21
4   Carpet      7  2007        6
5   Carpet      9  2006       18
6   Carpet     10  2004      110
7   Carpet     11  2003       47
8   Carpet     12  2006        4
9     Clay      1  2008       30
10    Clay      2  2010      120
11    Clay      3  2010       39
12    Clay      4  2010      273
13    Clay      5  2010      318
14    Clay      6  2008       31
15    Clay      7  2010      212
16    Clay      8  2008       30
17    Clay      9  2010       64
18    Clay     10  2010       14
19    Clay     11  2012        3
20    Clay     12  2006        4
21   Grass      2  2009        4
22   Grass      3  2014        6
23   Grass      4  2007        6
24   Grass      6  2010      259
25   Grass      7  2010       54
26   Grass      9  2009        4
27   Grass     11  2002        4
28    Hard      1  2010    

# Problem 5

Determine the BEST 5 players of all time.  There is not a definative answer here, this is your chance to show your creativity.  Please also explain how you arrived at your rankings.  You are free to use web resources to support your answer, but you MUST cite them as you use them.

You should answer questions like:

1. How did you define BEST?
2. Where do you believe your analysis is flawed?
3. What could you do to improve your analysis?

#### My Step 1: Determine Rank of tourney_level
According to Bing and Professor Natarajan, it is a widely known tennis fact that:
a. Higher drawsize tournaments (e.g., Grand Slams) involve more players and rounds. These tournaments attract top-ranked players, making the competition fierce.
b Lower drawsize tournaments (e.g., smaller ATP or WTA events) have fewer players, and the overall field is less competitive.

I know the rank for draw_size (i.e., higher draw size = higher rank), but I don't have information for how to rank tourney_level. Therefore, I rank tourney level according to its correlation with draw size

In [18]:
# Determine rank of tourney_level by finding the correlation between tourney_level and draw_size
# Filter the df for those two columns, and find the median draw_size for each tourney_level
df_filtered = df.filter(['draw_size', 'tourney_level']).sort_values(by=['tourney_level'])                                                 # filter the df
df_draw_size_median = df_filtered.groupby(['tourney_level']).median().astype(int).sort_values(by=['draw_size'], ascending=False).reset_index()   # find median draw size per tourney level
display(df_draw_size_median)

# Use the rankings obtained above to add a facotred tourney_level column to a filtered df
df_wtd = df.filter(["winner_id", "winner_name", "loser_id", "loser_name", "draw_size", "tourney_level"]).fillna(0)

# Factorize the 'tourney_level' column
factorized_values, unique_levels = pd.factorize(df_wtd['tourney_level'])

# Create a dictionary mapping original levels to specific values
mapping = {
    'G': 2,
    'M': 1.8,
    'A': 1.6,
    'F': 1.2,
    'D': 1
}

# Map factorized values to specific values using the mapping dictionary
df_wtd['tourney_level_factor'] = [mapping[unique_levels[i]] for i in factorized_values]
display(df_wtd.sort_values(by=['tourney_level_factor']))

,tourney_level,draw_size
0,G,128
1,M,64
2,A,32
3,F,8
4,D,4


,winner_id,winner_name,loser_id,loser_name,draw_size,tourney_level,tourney_level_factor
1867,104112,Benjamin Balleret,108734,Fredrik Ask,4,D,1.0
3065,104592,Ilija Bozoljac,105159,Vaja Uzakov,4,D,1.0
3064,104678,Viktor Troicki,104454,Murad Inoyatov,4,D,1.0
3063,104678,Viktor Troicki,104736,Farrukh Dustov,4,D,1.0
3062,104386,Janko Tipsarevic,104797,Denis Istomin,4,D,1.0
...,...,...,...,...,...,...,...
2441,103909,Guillermo Coria,102434,Vincent Spadea,128,G,2.0
2440,103813,Jarkko Nieminen,103451,Bjorn Phau,128,G,2.0
2439,103018,Max Mirnyi,103722,Florent Serra,128,G,2.0
2450,104607,Tomas Berdych,103672,Ricardo Mello,128,G,2.0


#### My Step 2: Compile a Data Frame of Players with Weighted Wins and Losses
Using the previously obtained data frame, I compile a data frame of all the winners and the losers. I create a joint data frame of all the players by counting up all their won and lost games, weighting the games by tourney_level_factor. I add a column called points, which is each players wins minus some fraction of their losses. Then, I choose the BEST 5 players as those with the most points.

(I had an I idea to compile a data frame of all the winners AND all the losers. Then join the win and lose dfs, and find the win and lose percentage for each player given the weighted game counts. Then the players with the highest win percentage are the best players. However, I found this to be an unfair estimate of BEST since if a player only played 1 game and won that 1 game, their win percentage woulld be 100. Nevertheless, I included this work below.)

In [20]:
# Create a table of all the wins
df_wtd_winners = df_wtd.filter(['winner_id', 'winner_name', 'tourney_level', 'tourney_level_factor'])
df_wtd_winners = df_wtd_winners.groupby(['winner_id', 'winner_name'])['tourney_level_factor'].sum().reset_index()
df_wtd_winners.rename(columns={'winner_id':'id', 'winner_name':'name', 'tourney_level_factor':'wins'}, inplace=True)
df_wtd_winners.sort_values(by=['wins'], ascending=False, inplace=True, ignore_index=True)
# print("df_wtd_winners")
# display(df_wtd_winners.head())

# Create a table of all the losses
df_wtd_losses = df_wtd.filter(['loser_id', 'loser_name', 'tourney_level', 'tourney_level_factor'])
df_wtd_losses = df_wtd_losses.groupby(['loser_id', 'loser_name'])['tourney_level_factor'].sum().reset_index()
df_wtd_losses.rename(columns={'loser_id':'id', 'loser_name':'name', 'tourney_level_factor':'losses'}, inplace=True)
# df_wtd_losses.sort_values(by=['losses'], ascending=True, inplace=True, ignore_index=True)
# print("df_wtd_losses")
# display(df_wtd_losses)

# Create a joint table of all players with their win and lose stats
df_wtd_players = df_wtd_winners.merge(df_wtd_losses, 'outer').fillna(0)
df_wtd_players['points'] = df_wtd_players['wins'] - (df_wtd_players['losses']/4)
df_wtd_players.sort_values(by=['points'], ascending=False, inplace=True, ignore_index=True)
print("My BEST Players")
display(df_wtd_players.head())

My BEST Players


,id,name,wins,losses,points
0,103819,Roger Federer,2162.2,442.2,2051.65
1,104745,Rafael Nadal,1789.2,362.0,1698.70
2,104925,Novak Djokovic,1670.8,332.2,1587.75
3,104918,Andy Murray,1191.2,357.8,1101.75
4,103970,David Ferrer,1260.8,647.4,1098.95


#### My Step 3: Compare to Original Rankings
To see how well my ranking did, I compare to the original df rankings, choosing the most recent rank for each player.
Then I compare the differences.

In [21]:
# Compare to original df ranking
# Find the players with the top rank, filtering out duplicate players by only keeping their rank from the most current date
df_winners = df.filter(['winner_id', 'winner_name', 'winner_rank', 'year', 'month']).sort_values(by=['year', 'month', 'winner_rank'], ascending=[False, False, True])
df_winners = df_winners.drop_duplicates(subset=['winner_name', 'winner_id'])
df_winners.rename(columns={'winner_id':'id', 'winner_name':'name', 'winner_rank':'rank'}, inplace=True)
df_winners.sort_values(by=['rank'], ascending=True, inplace=True, ignore_index=True)
print("Original Top Ranked Players")
display(df_winners.head())

# Of my BEST players, the Original Top Ranked Players is missing David Ferrer and Andy Murray. 
# Of the original Top Ranked Players, my BEST players are missing Dominic Thiem and Daniil Medvedev.
# Let's find where they fell in each data frame
print("Other Top Ranked Players Weighted Scores")
display(df_wtd_players[df_wtd_players['name'].isin(['Dominic Thiem', 'Daniil Medvedev'])])
print("My Other BEST Players Original Ranks")
display(df_winners[df_winners['name'].isin(['Andy Murray', 'David Ferrer'])])

Original Top Ranked Players


,id,name,rank,year,month
0,104925,Novak Djokovic,1.0,2020,11
1,104745,Rafael Nadal,2.0,2020,11
2,103819,Roger Federer,3.0,2020,1
3,106233,Dominic Thiem,3.0,2020,11
4,106421,Daniil Medvedev,4.0,2020,11


Other Top Ranked Players Weighted Scores


,id,name,wins,losses,points
44,106233,Dominic Thiem,515.2,268.0,448.20
101,106421,Daniil Medvedev,268.4,143.8,232.45


My Other BEST Players Original Ranks


,id,name,rank,year,month
233,104918,Andy Murray,115.0,2020,8
356,103970,David Ferrer,144.0,2019,5


#### My Step 4: Conclusion
I think my rankings did pretty well. In finding the BEST players I take into account tourney level, draw size, wins, and losses. My top 3 BEST players were consistent with the original top 3 highest ranked players.
I believe my analysis may be flawed because there is probably a better way to take into account losses in the determination of BEST.I would want tto improvemyr analysi by factoring in losses better, though I don't know how I would do that. Just looking at the wins and losses data I think my analysis does a fine job of finding the BEST players.?

#### Scrap Code

RANKING SURFACES

Another idea I had of how to decide 'BEST' is to take into account surface, but after research I found that it doesn't really contribute. I have kept my work below.

https://matchpointpost.com/best-tennis-court-surface/

https://www.tennisletics.com/blog/how-different-types-of-court-impact-a-tennis-match/

In [22]:
data = {'surface':   ['Carpet', 'Clay', 'Grass', 'Hard'],
        'speed':     ['fast', 'slow', 'fastest', 'medium'],
        'bounce' :   ['low', 'high', 'low', 'medium']}

surfaces = pd.DataFrame(data)

FAULTY REASONING WIN PERCENTAGE CODE

In [23]:
# Create a table of all the wins
df_wins = df_wtd.filter(['winner_id', 'winner_name', 'tourney_level', 'tourney_level_factor'])
df_wins = df_wins.groupby(['winner_id', 'winner_name'])['tourney_level_factor'].sum().reset_index()
df_wins.rename(columns={'winner_id':'id', 'winner_name':'name', 'tourney_level_factor':'wins'}, inplace=True)
# print("df_wins")
# display(df_wins.sort_values(by=['wins'], ascending=False))

# Create a table of all the losses
df_losses = df_wtd.filter(['loser_id', 'loser_name', 'tourney_level', 'tourney_level_factor'])
df_losses = df_losses.groupby(['loser_id', 'loser_name'])['tourney_level_factor'].sum().reset_index()
df_losses.rename(columns={'loser_id':'id', 'loser_name':'name', 'tourney_level_factor':'losses'}, inplace=True)
# print("df_losses")
# display(df_losses.sort_values(by=['losses'], ascending=False))

# Create a joint table of all players with their win and lose stats
# add a column of win percentage and lose percentage
df_players = df_wins.merge(df_losses, 'outer', sort=True).fillna(0)
df_players['games'] = df_players['wins'] + df_players['losses']
df_players['win_prc'] = (df_players['wins'] / df_players['games'] * 100).round(0).astype(int)
df_players['lose_prc'] = (df_players['losses'] / df_players['games'] * 100).round(0).astype(int)
df_players.sort_values(by='win_prc', ascending=False, inplace=True)
# print("df_players")
# display(df_players)

JUNK DRAWER

In [24]:
# print(df_tl_ds.to_string())

# to get the amount of draw_size games per tourney level
#df_tl_ds_size = df_tl_ds.groupby(['draw_size', 'tourney_level']).size().reset_index(name='count').sort_values(by=['tourney_level', 'draw_size'])
  
# display(df_wins.head())
# display(df_losses.head())
# print("df_players")
# display(df_players.head())
# df_players.info()

# df.info()
# temp2 = temp.groupby(["tourney_level", "winner_rank"]).size().reset_index(name="count")
# display(temp, temp2)
# print(temp.to_string())
# temp2.groupby(["tourney_level"]).nsmallest(5).reset_index()
# temp2.nlargest(5, "winner_rank", keep='all').sort_values(by=['tourney_level'])
# df_tl_ds = df.filter(["draw_size", "tourney_level", "winner_rank", "winner_name", "minutes"]).sort_values(by=['draw_size'])